## Q1. Running Elastic

In [93]:
!curl localhost:9200

{
  "name" : "4eec8250e5f1",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "xp6AeOj2QQyFxwdFxc3B8w",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [94]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [95]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [96]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}, # This field is of type "keyword" since it's used for filtering
        }
    },
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [97]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [118]:
import requests

docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [98]:
query_1 = "How do execute a command on a Kubernetes pod?"

In [ ]:
import json

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_1,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
        }
    },
}

response = es_client.search(index=index_name, body=search_query)

print(json.dumps(response.body, indent=4))

{
    "took": 160,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 739,
            "relation": "eq"
        },
        "max_score": 44.50556,
        "hits": [
            {
                "_index": "course-questions",
                "_id": "6FpTbpcBvGROU1vkl-PN",
                "_score": 44.50556,
                "_source": {
                    "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
                    "section": "5. Deploying Machine Learning Models",
                    "question": "How do I debug a docker container?",
                    "course": "

In [100]:
# Score for top ranking result
response['hits']['hits'][0]['_score']

44.50556

In [101]:
# Alternatively
response["hits"]["max_score"]

44.50556

## Q4. Filtering

In [102]:
# Q4
query_2 = "How do copy a file to a Docker container?"

In [103]:
import json

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_2,
                    "fields": ["question^4", "text"],
                    "type": "best_fields",
                }
            },
            "filter": {"term": {"course": "machine-learning-zoomcamp"}},
        }
    },
}

response = es_client.search(index=index_name, body=search_query)

print(json.dumps(response.body, indent=4))

{
    "took": 30,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 340,
            "relation": "eq"
        },
        "max_score": 73.38676,
        "hits": [
            {
                "_index": "course-questions",
                "_id": "6FpTbpcBvGROU1vkl-PN",
                "_score": 73.38676,
                "_source": {
                    "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
                    "section": "5. Deploying Machine Learning Models",
                    "question": "How do I debug a docker container?",
                    "course": "m

In [104]:
# 3rd question
response["hits"]["hits"][2]["_source"]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [ ]:
# Q5
result_docs = []

for hit in response["hits"]["hits"]:
    result_docs.append(hit["_source"]) 

result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [106]:
context_template = """
Q: {question}
A: {text}
""".strip()

print(context_template)
context_template

Q: {question}
A: {text}


'Q: {question}\nA: {text}'

In [107]:
context = ""

for doc in result_docs:
    context += f"Q: {doc['question']}\nA: {doc['text']}\n\n"

print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [110]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query_2, context=context).strip()

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [112]:
len(prompt)

1446

## Q6. Tokens

In [113]:
import tiktoken

In [114]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [119]:
prompt_tokens = encoding.encode(prompt)
print(prompt_tokens)

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 5150, 261, 1974, 316, 261, 91238, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480, 734, 1385, 5150, 261, 197

In [120]:
len(prompt_tokens)

320

## Generating the answer with Gemini

In [123]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-preview-05-20", contents=[prompt]
)
print(response.text)

To copy a file to a Docker container, you can use the `docker cp` command.

The basic syntax is as follows:
`docker cp /path/to/local/file_or_directory container_id:/path/in/container`
